In [1]:
# load general module
from __future__ import print_function
from keras.optimizers import SGD
import numpy as np
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import load_model
import h5py
from keras.layers import Dense, Convolution2D, BatchNormalization, MaxPooling2D, Input, Flatten
from keras.models import Sequential
from keras.utils.layer_utils import layer_from_config
import keras.backend as K
import copy
import matplotlib.pyplot as plt



# load data, prepare for training

# settings
batch_size = 32
nb_classes = 10
nb_epoch = 200
data_augmentation = True
trained = False

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.
X_test /= 255.
X_train -= np.mean(X_train, axis=0)
X_test -= np.mean(X_test, axis=0)


print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)





Using TensorFlow backend.


X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 25 10:43:17 2017

@author: moyan
"""
from __future__ import print_function
import numpy as np
import h5py
from keras.models import Sequential, Model
import keras.backend as K
import copy
from keras.layers import Dense, Input, Flatten, BatchNormalization, Convolution2D, MaxPooling2D




def get_network():
    """
    return a vgg-like cnn for cifar10 classification, not trained.
    """
    img_input = Input(shape=(32,32,3))
    x = Convolution2D(64, 3, 3, activation='relu', border_mode= 'same', name='block1_conv1')(img_input)
    x = BatchNormalization(axis=3, name='block1_bn1')(x)
    x = Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv2')(x)
    x = BatchNormalization(axis=3, name='block1_bn2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv1')(x)
    x = BatchNormalization(axis=3, name='block2_bn1')(x)
    x = Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv2')(x)
    x = BatchNormalization(axis=3, name='block2_bn2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv1')(x)
    x = BatchNormalization(axis=3, name='block3_bn1')(x)
    x = Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv2')(x)
    x = BatchNormalization(axis=3, name='block3_bn2')(x)
    x = Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv3')(x)
    x = BatchNormalization(axis=3, name='block3_bn3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv1')(x)
    x = BatchNormalization(axis=3, name='block4_bn1')(x)
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv2')(x)
    x = BatchNormalization(axis=3, name='block4_bn2')(x)
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv3')(x)
    x = BatchNormalization(axis=3, name='block4_bn3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv1')(x)
    x = BatchNormalization(axis=3, name='block5_bn1')(x)
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv2')(x)
    x = BatchNormalization(axis=3, name='block5_bn2')(x)
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv3')(x)
    x = BatchNormalization(axis=3, name='block5_bn3')(x)

    x = Flatten(name='flatten')(x)
    x = Dense(512, activation='relu', name='fc1')(x)
    x = BatchNormalization(axis=1, name='fc_bn1')(x)
    x = Dense(10, activation='softmax', name='predictions')(x)

    model = Model([img_input], [x])
    return model


def MoveNetwork(net, layer_before_flatten=None):
    """
    This function will extract a subnetwork from the original network, only work with plain networks.
    A plain network is a network with only one input and one output, contains only:
        - Convolution2D
        - MaxPooling2D/AveragePooling2D
        - BatchNormalization
        - Dropout
        - Flatten
        - Dense
        - Activation
    Neurons whose parameters contain at least a non-zero value would be extracted.

    params:
        - net: A Keras model
        - layer_before_flatten: str, the name of the convolution layer just before the flatten layer.
    returns:
        - A sub-network extracted from the original one.
    """
    model = Sequential()
    weight_list = []
    for layer in net.layers:
        if layer.__class__.__name__ == "InputLayer":
            continue
        if layer.__class__.__name__ == 'Convolution2D':
            if len(model.layers)==0:
                w,b = layer.get_weights()
                save_id = []
                for fid in range(w.shape[3]):
                    if np.sum(np.abs(w[:,:,:,fid]))!=0.:
                        save_id.append(fid)
                w_new = np.zeros(w.shape[:3]+(len(save_id),))
                b_new = np.zeros(len(save_id))

                i = 0
                for fid in save_id:
                    w_new[:,:,:,i] = w[:,:,:,fid]
                    b_new[i] = b[fid]
                    i+=1
                config = layer.get_config()
                new_layer = Convolution2D(len(save_id), config['nb_col'], config['nb_row'],
                                          activation=config['activation'], border_mode=config['border_mode'],
                                          name=config['name'], input_shape=(32,32,3))
                weight_list.append([w_new, b_new])
                model.add(new_layer)
                prev_save_id = copy.deepcopy(save_id)
            else:
                nb_channels = len(prev_save_id)
                w,b = layer.get_weights()
                save_id = []
                for fid in range(w.shape[3]):
                    if np.sum(np.abs(w[:,:,:,fid]))>0.0000001:
                        save_id.append(fid)

                w_new = np.zeros(w.shape[:2]+(nb_channels,len(save_id)))
                b_new = np.zeros(len(save_id))

                i = 0
                for fid in save_id:
                    w_new[:,:,:,i] = w[:,:,prev_save_id,fid]
                    b_new[i] = b[fid]
                    i+=1
                config = layer.get_config()
                new_layer = Convolution2D(len(save_id), config['nb_col'], config['nb_row'],
                          activation=config['activation'], border_mode=config['border_mode'],
                          name=config['name'])
                weight_list.append([w_new, b_new])
                model.add(new_layer)
                prev_save_id = copy.deepcopy(save_id)

        elif layer.__class__.__name__ == 'BatchNormalization':
            gamma, beta, mean, std = layer.get_weights()
            nb_channels = len(prev_save_id)
            gamma_new = np.zeros(len(prev_save_id))
            beta_new = np.zeros(len(prev_save_id))
            mean_new = np.zeros(len(prev_save_id))
            std_new = np.zeros(len(prev_save_id))


            gamma_new = gamma[prev_save_id]
            beta_new = beta[prev_save_id]
            mean_new = mean[prev_save_id]
            std_new = std[prev_save_id]


            config = layer.get_config()
            new_layer = BatchNormalization(name=config['name'], axis=config['axis'])
            weight_list.append([gamma_new, beta_new, mean_new, std_new])
            model.add(new_layer)
            
        elif layer.__class__.__name__ == 'MaxPooling2D' or layer.__class__.__name__ == 'AveragePooling2D':
            config = layer.get_config()
            new_layer = MaxPooling2D(pool_size = config['pool_size'], strides = config['pool_size'], name = config['name'])
            model.add(new_layer)
            weight_list.append(None)

        elif layer.__class__.__name__=='Flatten':
            new_layer = Flatten(name = 'flatten')
            model.add(new_layer)
            weight_list.append(None)

        elif layer.__class__.__name__=='Dense':
            if model.layers[-1].__class__.__name__=='Flatten':
                last_layer = net.get_layer(layer_before_flatten)
                last_layer_output_shape = K.int_shape(last_layer.output)[1:]
                fake_input = np.zeros(last_layer_output_shape)
                fake_input[:,:,prev_save_id] = 1.
                fake_input = np.expand_dims(fake_input, axis=0)

                input_tensor = K.placeholder(fake_input.shape)
                output_tensor = Flatten()(input_tensor)
                get_indicator = K.function([input_tensor, K.learning_phase()],
                          [output_tensor])
                indicators = get_indicator([fake_input, 0])[0]
                indicators = np.squeeze(indicators)
                indicator_index = []
                for i in range(indicators.shape[0]):
                    if indicators[i]==1.:
                        indicator_index.append(i)
                save_id = []
                w,b = layer.get_weights()
                for i in range(w.shape[1]):
                    if np.sum(np.abs(w[:,i]))!=0.:
                        save_id.append(i)
                w_new = w[indicator_index,:]
                w_new = w_new[:,save_id]
                b_new = b[save_id]
                config = layer.get_config()
                new_layer = Dense(len(save_id), activation=config['activation'], name=config['name'])
                weight_list.append([w_new,b_new])
                model.add(new_layer)
                prev_save_id = copy.deepcopy(save_id)

            else:
                w,b = layer.get_weights()
                w_new = w[prev_save_id,:]
                config = layer.get_config()
                new_layer = Dense(config['output_dim'], activation=config['activation'], name=config['name'])
                weight_list.append([w_new,b])
                model.add(new_layer)

        elif layer.__class__.__name__=='Dropout':
            config = layer.get_config()
            new_layer = Dropout(config['p'])
            weight_list.append(None)
            model.add(new_layer)

            
        else:
            print("Exist unknown layer type:%s"%layer.name)


    for i in range(len(model.layers)):
        if weight_list[i] == None:
            continue
        model.layers[i].set_weights(weight_list[i])
    return model


def next_layer(target_layer):
    """
    This function is set to find the BatchNormalization after the conv layers and fc layers
    you may always need to define this function if your network contains BatchNormalization layer

    """
    next_layers={"block1_conv1":"block1_bn1",
               "block1_conv2":"block1_bn2",
               "block2_conv1":"block2_bn1",
               "block2_conv2":"block2_bn2",
               "block3_conv1":"block3_bn1",
               "block3_conv2":"block3_bn2",
               "block3_conv3":"block3_bn3",
               "block4_conv1":"block4_bn1",
               "block4_conv2":"block4_bn2",
               "block4_conv3":"block4_bn3",
               "block5_conv1":"block5_bn1",
               "block5_conv2":"block5_bn2",
               "block5_conv3":"block5_bn3",
               "fc1":"fc_bn1"
          }
    return next_layers[target_layer]



def get_activation(model, prefix, X_train, y_train):
    """
    This function obtain the activation of each layer when using data-dependent neuron selection strategy and
    save them in several hdf5 files.
    params:
        model: the network
        prefix: the name
    """

    for target_layer in model.layers:
        if target_layer.__class__.__name__ in ["Convolution2D", "Dense"]:
            input_tensor = model.input
            output_tensor = target_layer.output

            get_outputs = K.function([input_tensor, K.learning_phase()],
                              [output_tensor])

            raw_stat = {}
            for s in range(X_train.shape[0]):
                img = np.expand_dims(X_train[s], axis=0)
                label = y_train[s]
                output = get_outputs([img, 0])[0]
                output = np.squeeze(output)
                if target_layer.__class__.__name__=='Dense':
                    output = output
                elif target_layer.__class__.__name__=='Convolution2D':
                    output = np.mean(output, axis = (0,1))
                assert(len(output.shape)==1)

                if raw_stat.has_key(int(label)):
                    raw_stat[int(label)].append(output)
                else:
                    raw_stat[int(label)]= [output]


            f = h5py.File(save_name,'w')
            for k in raw_stat.keys():
                f.create_dataset(name=str(k), data=raw_stat[k])
            f.close()


class NetScore(object):
    """
    This class record the contribution score of all neurons in network. And provide a series of
    methods to get the redundant neurons.

    You may set some properties when instantiating.

    protect_dict: dict, mapping layer_name -> nb_neurons to be protected. If given, at least 'protect_dict[layer_name]'' neurons
                  at 'layer_name' would be preserved.
    model: str, "global" or "layer_wise", default "global". If set to "layer_wise", the number of redundant neurons in each layer
                will be determined by a same pruning ratio or the number given by a dict.
                if set to "global", the number of redundant neurons in each layer will be determined according to the score.

    The methods are:

        add(layer_name, neuron_id, score): add an neuron in the neuron list.
            params:
                layer_name: str, the name of layer that contains the neuron
                neuron_id: int, the index of the neuron in this layer
                score: float, the contribution score of the layer

        sort(): sort the neurons according the contribution score.
        get_redundant_neurons(p): return redundant neurons
            params:
                p: float or dict, if dict, the mode must be layer_wise. if float(0.0~1.0), in layer_wise mode, the redundant neurons
                   in each layer will be selected according to this parameter. In "global" mode, this is the overall pruning ratio
                   of the whole network.
        print_info(): print infos.
    """

    def __init__(self, protect_dict=None, mode="global"):
        self.netscores = {}
        self.sorted = False
        self.total_num = 0
        self.protected_num = 0
        self.neuron_num = {}
        self.prune_num = {}
        self.protect_dict = protect_dict
        self.mode = mode
        self.redundant = None

    def add(self, layer_name, neuron_id, score):
        if self.netscores.has_key(layer_name):
            self.netscores[layer_name].append((layer_name, neuron_id, score))
            self.neuron_num[layer_name] += 1
        else:
            self.netscores[layer_name] = [(layer_name, neuron_id, score)]
            self.neuron_num[layer_name] = 1

        self.total_num += 1

    def sort(self):
        if self.mode == "layer_wise":
            for k in self.netscores.keys():
                self.netscores[k] = sorted(self.netscores[k], key=lambda x:x[2])
            self.sorted = True
        elif self.mode == "global":
            netscores = []
            if self.protect_dict != None:
                for k in self.netscores.keys():
                    self.netscores[k] = sorted(self.netscores[k], key=lambda x:x[2])
                    self.netscores[k] = self.netscores[k][:-self.protect_dict[k]]
            for k in self.netscores.keys():
                netscores += self.netscores[k]
            self.netscores = sorted(netscores, key=lambda x:x[2])
            self.protected_num = self.total_num - len(self.netscores)

    def get_redundant_neurons(self, p):
        if isinstance(p, float):
            if self.mode == "layer_wise":
                if not self.sorted:
                    self.sort()
                redundant = []
                print(type(self.netscores))
                print(type(self.netscores.keys()))
                for k in self.netscores.keys():
                    redundant += self.netscores[k][:np.round(self.neuron_num[k]*p)]

                self.redundant = redundant
            elif self.mode == "global":
                self.redundant = self.netscores[:int(np.round(self.total_num*p))]

        if isinstance(p, dict):
            if self.mode == "layer_wise":
                if not self.sorted:
                    self.sort()
                redundant = []

                for k in self.netscores.keys():
                    redundant += self.netscores[k][:p[k]]
                self.redundant = redundant
            elif self.mode == "global":
                raise TypeError("dict only work for global mode!")
                    

        return self.redundant

    def print_info(self):
        for neuron in self.redundant:
            layer_name, neuron_id, score = neuron
            if self.prune_num.has_key(layer_name):
                self.prune_num[layer_name] += 1
            else:
                self.prune_num[layer_name] = 1
        if self.protect_dict == None:
            for k in self.prune_num.keys():
                print("%s: pruned:%d, protected:%d"%(k, self.prune_num[k], 0))
        else:
            for k in self.prune_num.keys():
                print("%s: pruned:%d, protected:%d"%(k, self.prune_num[k], self.protect_dict[k]))
        
        print("pruning: %d -> %d, %d get pruned"%(self.total_num, self.total_num-len(self.redundant), len(self.redundant)))


def get_num_by_ratio(neuron_num,p):
    """
    this function returns the dictionary according to pruning ratio.
    this function should use with mode="layer_wise" and the case when p is a dict.
    We use np.round to erase the errors introduced by integer division
    """
    total_num = 0
    for k in neuron_num:
        total_num += neuron_num[k]
    total_pruned_num = np.round(total_num*p)
    for k in neuron_num:
        neuron_num[k] = int(np.round(total_pruned_num*(neuron_num[k]/float(total_num))))
    return neuron_num



In [ ]:

method = "AAWS"
mode = "global"
#pruning_ratio = 0.03
for pruning_ratio in [0.05]:
    prune_round = 1
    target_layers = ["block1_conv1","block1_conv2",
                   "block2_conv1","block2_conv2",
                   "block3_conv1","block3_conv2","block3_conv3",
                   "block4_conv1","block4_conv2","block4_conv3",
                   "block5_conv1","block5_conv2","block5_conv3","fc1"]

    while True:
        model_name = "vgg_cifar10_"+str(prune_round-1)+".h5"
        model = load_model(model_name)
        
        protect_dict = {"block1_conv1":32,
                        "block1_conv2":32,
                        "block2_conv1":64,
                        "block2_conv2":64,
                        "block3_conv1":128,
                        "block3_conv2":128,
                        "block3_conv3":128,
                        "block4_conv1":256,
                        "block4_conv2":256,
                        "block4_conv3":256,
                        "block5_conv1":256,
                        "block5_conv2":256,
                        "block5_conv3":256,
                        "fc1":100}
        netscore = NetScore(mode=mode)

        if method == 'AAWS':
            for target_layer in target_layers:
                if target_layer[:2]=='fc':
                    w,b = model.get_layer("predications").get_weights()
                    weight_sum = np.mean(np.abs(w), axis=1)
                    weight_sum = weight_sum/np.mean(weight_sum)
                else:
                    w,b = model.get_layer(target_layer).get_weights()
                    weight_sum = np.mean(np.abs(w), axis=(0,1,2))
                    weight_sum = weight_sum/np.mean(weight_sum)

                for neuron_id in range(weight_sum.shape[0]):
                    netscore.add(target_layer, neuron_id, weight_sum[neuron_id])
        if mode=='layer_wise':
            neuron_num = netscore.neuron_num
            p = get_num_by_ratio(neuron_num, pruning_ratio)

        else:
            p = pruning_ratio
            
        netscore.sort()
        redundant_neurons = netscore.get_redundant_neurons(p)


        for neuron in redundant_neurons:
            neuron_layer, neuron_id, neuron_score = neuron
            if neuron_layer[:2]=='fc':
                w,b = model.get_layer("predications").get_weights()
                w[neuron_id,:] = 0.
                model.get_layer("predications").set_weights([w,b])

                w,b = model.get_layer(neuron_layer).get_weights()
                w[:, neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])

            else:
            # do pruning to target_layer
                w,b = model.get_layer(neuron_layer).get_weights()
                w[:,:,:,neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])
            # assign bn
            bn_gamma, bn_beta, bn_mean, bn_std = model.get_layer(next_layer(neuron_layer)).get_weights()
            bn_gamma[neuron_id] = 0.
            bn_beta[neuron_id] = 0.
            bn_mean[neuron_id] = 0.
            bn_std[neuron_id] = 1.
            model.get_layer(next_layer(neuron_layer)).set_weights([bn_gamma, bn_beta, bn_mean, bn_std])

        netscore.print_info()
        model = MoveNetwork(model, "block5_conv3")



    # fine-tune

        model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        score = model.evaluate(X_test, Y_test, verbose=0)
        print("Moved Model ACC: %f"%score[1])
        model.fit(X_train, Y_train,batch_size=50,nb_epoch=1,validation_data=(X_test, Y_test),shuffle=True)

        sgd = SGD(lr=0.01, momentum=0.9, decay=5*1e-4,
                     nesterov=True)
        model.compile(loss = 'categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        model.fit(X_train, Y_train,batch_size=50,nb_epoch=1,validation_data=(X_test, Y_test),shuffle=True)

        model.save("vgg_cifar10_"+str(prune_round)+".h5")

        fine_score = model.evaluate(X_test, Y_test, verbose=0)
        print("finetuned_acc: %f"%fine_score[1])
        record = "round:%d nb_neurons:%d acc:%f fintune_acc:%f\n"%(prune_round, netscore.total_num, score[1],fine_score[1])
        print(record)
        print("------------------------------")
        prune_round+=1
        if fine_score[1]<0.5:
            break

fc1: pruned:230, protected:0
block1_conv1: pruned:6, protected:0
block5_conv2: pruned:1, protected:0
pruning: 4736 -> 4499, 237 get pruned
Moved Model ACC: 0.873800
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 197s - loss: 0.0045 - acc: 0.9986 - val_loss: 0.8405 - val_acc: 0.8687
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 189s - loss: 0.0026 - acc: 0.9991 - val_loss: 0.7868 - val_acc: 0.8740
finetuned_acc: 0.874000
round:1 nb_neurons:4736 acc:0.873800 fintune_acc:0.874000

------------------------------
fc1: pruned:140, protected:0
block4_conv3: pruned:18, protected:0
block5_conv3: pruned:16, protected:0
block5_conv2: pruned:33, protected:0
block5_conv1: pruned:17, protected:0
block1_conv1: pruned:1, protected:0
pruning: 4499 -> 4274, 225 get pruned
Moved Model ACC: 0.873900
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [===============

In [ ]:
## no fine-tune

mode = "layer_wise"
for method in ["AAWS"]:
    for pruning_ratio in [0.,0.02,0.04,0.06,0.08,0.10,0.12,0.14,0.16,0.18,0.20,0.22,0.24,0.26,0.28,0.30]:
        prune_round = 1
        target_layers = ["block1_conv1","block1_conv2",
                   "block2_conv1","block2_conv2",
                   "block3_conv1","block3_conv2","block3_conv3",
                   "block4_conv1","block4_conv2","block4_conv3",
                   "block5_conv1","block5_conv2","block5_conv3","fc1"]


        model_name = "vgg_cifar10_"+str(prune_round-1)+".h5"
        model = load_model(model_name)
        netscore = NetScore(mode=mode)

        if method == 'AAWS':
            for target_layer in target_layers:
                if target_layer[:2]=='fc':
                    w,b = model.get_layer("predications").get_weights()
                    weight_sum = np.mean(np.abs(w), axis=1)
                    weight_sum = weight_sum/np.mean(weight_sum)
                else:
                    w,b = model.get_layer(target_layer).get_weights()
                    weight_sum = np.mean(np.abs(w), axis=(0,1,2))
                    weight_sum = weight_sum/np.mean(weight_sum)

                for neuron_id in range(weight_sum.shape[0]):
                    netscore.add(target_layer, neuron_id, weight_sum[neuron_id])
        elif method == "mean" or method == "sigma":
            mean_dict = {}
            std_dict = {}
            all_data = {}
            for target_layer in target_layers:

                f = h5py.File("prune"+str(prune_round)+'_'+target_layer+'_cifar10.h5', 'r')
                raw_stat = {}

                for k in f.keys():
                    raw_stat[k] = f[k][:]


                all_data[target_layer] = raw_stat
                # put activation together
                act_data = []
                for k in raw_stat.keys():
                    act_data += list(raw_stat[k])

                act_data = np.array(act_data)

                mean_dict[target_layer] = np.mean(act_data, axis=0)
                std_dict[target_layer] = np.std(act_data, axis=0)
                mean_dict[target_layer] = mean_dict[target_layer]/np.mean(mean_dict[target_layer])
                std_dict[target_layer] = std_dict[target_layer]/np.mean(std_dict[target_layer])
        if method == 'sigma':
            for target_layer in target_layers:
                for neuron_id in range(std_dict[target_layer].shape[0]):
                    netscore.add(target_layer, neuron_id, std_dict[target_layer][neuron_id])

        elif method == 'mean':
            for target_layer in target_layers:
                for neuron_id in range(mean_dict[target_layer].shape[0]):
                    netscore.add(target_layer, neuron_id, mean_dict[target_layer][neuron_id])
                    
        if mode=='layer_wise':
            neuron_num = netscore.neuron_num
            p = get_num_by_ratio(neuron_num, pruning_ratio)

        else:
            p = pruning_ratio
            
        netscore.sort()
        redundant_neurons = netscore.get_redundant_neurons(p)


        for neuron in redundant_neurons:
            neuron_layer, neuron_id, neuron_score = neuron
            if neuron_layer[:2]=='fc':
                w,b = model.get_layer("predications").get_weights()
                w[neuron_id,:] = 0.
                model.get_layer("predications").set_weights([w,b])

                w,b = model.get_layer(neuron_layer).get_weights()
                w[:, neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])

            else:
            # do pruning to target_layer
                w,b = model.get_layer(neuron_layer).get_weights()
                w[:,:,:,neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])
            # assign bn
            bn_gamma, bn_beta, bn_mean, bn_std = model.get_layer(next_layer(neuron_layer)).get_weights()
            bn_gamma[neuron_id] = 0.
            bn_beta[neuron_id] = 0.
            bn_mean[neuron_id] = 0.
            bn_std[neuron_id] = 1.
            model.get_layer(next_layer(neuron_layer)).set_weights([bn_gamma, bn_beta, bn_mean, bn_std])

        netscore.print_info()
        
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        fine_score = model.evaluate(X_test, Y_test, verbose=0)
        print("finetuned_acc: %f"%fine_score[1])
        record = "ratio:%f nb_neurons:%d fintune_acc:%f\n"%(pruning_ratio, netscore.total_num,fine_score[1])
        print(record)
        print("------------------------------")

pruning: 4736 -> 4736, 0 get pruned
finetuned_acc: 0.873200
ratio:0.000000 nb_neurons:4736 fintune_acc:0.873200

------------------------------
fc1: pruned:10, protected:0
block2_conv1: pruned:3, protected:0
block2_conv2: pruned:3, protected:0
block4_conv2: pruned:10, protected:0
block4_conv3: pruned:10, protected:0
block4_conv1: pruned:10, protected:0
block5_conv3: pruned:10, protected:0
block5_conv2: pruned:10, protected:0
block5_conv1: pruned:10, protected:0
block1_conv2: pruned:1, protected:0
block1_conv1: pruned:1, protected:0
block3_conv1: pruned:5, protected:0
block3_conv3: pruned:5, protected:0
block3_conv2: pruned:5, protected:0
pruning: 4736 -> 4643, 93 get pruned
finetuned_acc: 0.862500
ratio:0.020000 nb_neurons:4736 fintune_acc:0.862500

------------------------------
fc1: pruned:20, protected:0
block2_conv1: pruned:5, protected:0
block2_conv2: pruned:5, protected:0
block4_conv2: pruned:20, protected:0
block4_conv3: pruned:20, protected:0
block4_conv1: pruned:20, protected:

In [2]:
import time
from keras.models import load_model
model_name = "vgg_cifar10_"+str(0)+".h5"
model = load_model(model_name)
model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
t1 = time.time()
fine_score = model.evaluate(X_test, Y_test, verbose=1)
t2 = time.time()
print(t2-t1)

model_name = "vgg_cifar10_"+str(7)+".h5"
model = load_model(model_name)
model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
t1 = time.time()
fine_score = model.evaluate(X_test, Y_test, verbose=1)
t2 = time.time()
print(t2-t1)

10000/10000 [==============================] - 14s    
14.1828458309
10000/10000 [==============================] - 7s     
7.8745071888
